In [2]:
import pandas as pd
import glob

def load_multiple_excel_files(folder_path):
    """
    Load multiple Excel files from a folder into a dictionary of DataFrames.
    
    Parameters:
        folder_path (str): Path to the folder containing Excel files.
    
    Returns:
        dict: A dictionary where keys are file names and values are DataFrames.
    """
    # Find all Excel files in the folder
    excel_files = glob.glob(f"{folder_path}/*.xlsx")  # Adjust to "*.xls" if needed
    
    # Initialize a dictionary to store DataFrames
    dataframes = {}
    
    for file in excel_files:
        # Extract file name without extension
        file_name = file.split('/')[-1].split('.')[0]
        
        # Load the Excel file into a DataFrame
        dataframes[file_name] = pd.read_excel(file)
    
    return dataframes

In [3]:
folder_path = "File path"# Replace with the folder containing your Excel files
loaded_dataframes = load_multiple_excel_files(folder_path)

# Access the DataFrames
for file_name, df in loaded_dataframes.items():
    print(f"File: {file_name}")
    print(df.head())  # Display the first few rows of each DataFrame

File: vader redo\absa_sentiments
        date                                               post
0 2024-11-20                                  Absa? Never again
1 2024-11-18  @AbsaGhana I want a loan to go and write my En...
2 2024-11-18  @AbsaGhana JobCenterGH offers a fast and effic...
3 2024-11-17  @AbsaGhana $AMB price dropped but $SWINE Buys ...
4 2024-11-16  @AbsaGhana Imitating Access Bank method! Great...
File: vader redo\access_bank_sentiments
        date                                               post
0 2024-11-19  Can you people send give me my money ? @moneyg...
1 2024-11-19            I bought 21gh airtime, hasn’t reflected
2 2024-11-18  Reporting a fraudster who is deceiving people ...
3 2024-11-15   @AccessBankGhana , I’ve spent close to 2 hour...
4 2024-11-12   @AccessBankGhana please can you work on my na...
File: vader redo\adb_sentiments
        date                                               post
0 2024-11-19   @adb_Ghana l have still not received a transa...

In [7]:
!pip install emoji
!pip install contractions

   ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
   ------------------------ --------------- 174.1/289.9 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 289.9/289.9 kB 4.5 MB/s eta 0:00:00


In [8]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from emoji import demojize
import contractions

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_multiple_dataframes(dataframes, column_name):
    """
    Preprocess a specified text column in a dictionary of DataFrames.
    
    Parameters:
        dataframes (dict): A dictionary of DataFrames (e.g., loaded from Excel sheets).
        column_name (str): The name of the text column to preprocess.
    
    Returns:
        dict: A dictionary where keys are the same as the input, and values are DataFrames 
              with an additional 'cleaned_<column_name>' column.
    """
    # Initialize a lemmatizer and stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    def preprocess_text(text):
        """
        Preprocess a single text entry by cleaning, tokenizing, removing stopwords, and lemmatizing.
        """
        if not isinstance(text, str):
            return ''
        
        
        # Normalize text
        text = text.lower()
        text = contractions.fix(text)

        # Remove URLs and email addresses
        text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
        text = re.sub(r'\S+@\S+', '', text)
    
        # Handle emojis
        text = demojize(text)
    
        # Remove special characters (retain punctuation and emojis)
        text = re.sub(r'[^A-Za-z0-9\s\!\?\.,:]', '', text)
    
        # Reduce repeated characters
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
        
        # Remove mentions, hashtags, URLs, and special characters
        text = re.sub(r'@\w+', '', text)  # Remove mentions
        text = re.sub(r'#\w+', '', text)  # Remove hashtags
        

        # Step 2: Tokenization
        tokens = word_tokenize(text)

        # Step 3: Remove stopwords
        tokens = [word for word in tokens if word not in stop_words]

        # Step 4: Lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Join tokens back into a single string
        return ' '.join(tokens)

    # Process each DataFrame in the dictionary
    processed_dataframes = {}
    for key, df in dataframes.items():
        # Ensure the column exists in the DataFrame
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in DataFrame '{key}'.")
        
        # Make a copy of the DataFrame to avoid modifying the original
        df_copy = df.copy()

        # Apply preprocessing to the specified column
        df_copy[f'cleaned_{column_name}'] = df_copy[column_name].apply(preprocess_text)
        
        # Store the processed DataFrame in the dictionary
        processed_dataframes[key] = df_copy
    
    return processed_dataframes


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
processed_dfs = preprocess_multiple_dataframes(loaded_dataframes, 'post')

In [10]:
for sheet_name, df in processed_dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.head())  # Display the first few rows of the processed DataFrame

Sheet: vader redo\absa_sentiments
        date                                               post  \
0 2024-11-20                                  Absa? Never again   
1 2024-11-18  @AbsaGhana I want a loan to go and write my En...   
2 2024-11-18  @AbsaGhana JobCenterGH offers a fast and effic...   
3 2024-11-17  @AbsaGhana $AMB price dropped but $SWINE Buys ...   
4 2024-11-16  @AbsaGhana Imitating Access Bank method! Great...   

                                        cleaned_post  
0                                       absa ? never  
1  absaghana want loan go write energy commission...  
2  absaghana jobcentergh offer fast efficient hir...  
3  absaghana amb price dropped swine buy keep com...  
4  absaghana imitating access bank method ! great...  
Sheet: vader redo\access_bank_sentiments
        date                                               post  \
0 2024-11-19  Can you people send give me my money ? @moneyg...   
1 2024-11-19            I bought 21gh airtime, hasn’t refl

In [26]:
!pip install nltk

In [27]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer

def vader_sentiment_analysis(dataframes, text_column):
    """
    Perform sentiment analysis using VADER on the specified column of multiple DataFrames.

    Parameters:
        dataframes (dict): A dictionary of DataFrames.
        text_column (str): The name of the column containing cleaned text.

    Returns:
        dict: A dictionary of DataFrames with additional sentiment score columns.
    """
    # Initialize VADER sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    processed_dataframes = {}

    for key, df in dataframes.items():
        # Ensure the column exists
        if text_column not in df.columns:
            raise ValueError(f"Column '{text_column}' not found in DataFrame '{key}'.")

        # Copy DataFrame to avoid modifying the original
        df_copy = df.copy()

        # Apply VADER sentiment analysis
        df_copy['compound'] = df_copy[text_column].apply(lambda x: sia.polarity_scores(x)['compound'])
        df_copy['positive'] = df_copy[text_column].apply(lambda x: sia.polarity_scores(x)['pos'])
        df_copy['neutral'] = df_copy[text_column].apply(lambda x: sia.polarity_scores(x)['neu'])
        df_copy['negative'] = df_copy[text_column].apply(lambda x: sia.polarity_scores(x)['neg'])

        # Store the processed DataFrame
        processed_dataframes[key] = df_copy
    
    return processed_dataframes


In [12]:
# Run VADER sentiment analysis on the 'cleaned_post' column
vader_results = vader_sentiment_analysis(processed_dfs, 'cleaned_post')

# Access results
for sheet_name, df in vader_results.items():
    print(f"Sheet: {sheet_name}")
    print(df[['cleaned_post', 'compound', 'positive', 'neutral', 'negative']].head())


Sheet: vader redo\absa_sentiments
                                        cleaned_post  compound  positive  \
0                                       absa ? never    0.0000     0.000   
1  absaghana want loan go write energy commission...    0.2500     0.288   
2  absaghana jobcentergh offer fast efficient hir...    0.6249     0.338   
3  absaghana amb price dropped swine buy keep com...    0.6892     0.265   
4  absaghana imitating access bank method ! great...    0.6892     0.299   

   neutral  negative  
0    1.000     0.000  
1    0.593     0.119  
2    0.662     0.000  
3    0.735     0.000  
4    0.701     0.000  
Sheet: vader redo\access_bank_sentiments
                                        cleaned_post  compound  positive  \
0  people send give money ? moneygram made sent m...   -0.3400     0.054   
1                    bought 21gh airtime , reflected    0.0000     0.000   
2  reporting fraudster deceiving people job recru...   -0.4019     0.118   
3  accessbankghana , spent

In [13]:
import os

# Folder path where you want to save the files
output_folder = "File path"  # Replace with the actual path

# Ensure the folder exists, create it if it doesn't
os.makedirs(output_folder, exist_ok=True)

# Define a function to sanitize sheet names and file names
def sanitize_name(name):
    # Replace invalid characters with an underscore or remove them
    sanitized_name = re.sub(r'[\\/:*?"<>|]', '_', name)
    return sanitized_name

# Loop through each DataFrame in the vader_results dictionary
for sheet_name, df in vader_results.items():
    # Sanitize the sheet name to remove invalid characters
    sanitized_sheet_name = sanitize_name(sheet_name)
    
    # Construct the file path for each DataFrame (using sanitized sheet name)
    output_file_path = os.path.join(output_folder, f"{sanitized_sheet_name}_sentiment_results.xlsx")

    # Save the DataFrame to Excel with the sanitized sheet name
    df.to_excel(output_file_path, index=False, sheet_name=sanitized_sheet_name)

    print(f"Results for {sheet_name} have been saved to {output_file_path}.")

Results for vader redo\absa_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_absa_sentiments_sentiment_results.xlsx.
Results for vader redo\access_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_access_bank_sentiments_sentiment_results.xlsx.
Results for vader redo\adb_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_adb_sentiments_sentiment_results.xlsx.
Results for vader redo\calbank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_calbank_sentiments_sentiment_results.xlsx.
Results for vader redo\cbg_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_cbg

C:\Users\User\.conda\envs\creating_an_environment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Results for vader redo\ecobank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_ecobank_sentiments_sentiment_results.xlsx.
Results for vader redo\fidelity_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_fidelity_bank_sentiments_sentiment_results.xlsx.
Results for vader redo\first_atlantic_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_first_atlantic_bank_sentiments_sentiment_results.xlsx.
Results for vader redo\gcb_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/redo sentiment results\vader redo_gcb_sentiments_sentiment_results.xlsx.
Results for vader redo\gtbank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/vader redo/

In [43]:
!pip install textblob

   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   - ------------------------------------- 20.5/626.3 kB 682.7 kB/s eta 0:00:01
   ---------- ----------------------------- 163.8/626.3 kB 2.5 MB/s eta 0:00:01
   ------------------------------------ --- 573.4/626.3 kB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 626.3/626.3 kB 4.9 MB/s eta 0:00:00


In [44]:
from textblob import TextBlob

def sentiment_analysis(dataframes, text_column):
    """
    Perform sentiment analysis on the specified column of multiple DataFrames.

    Parameters:
        dataframes (dict): A dictionary of DataFrames.
        text_column (str): The name of the column containing cleaned text.

    Returns:
        dict: A dictionary of DataFrames with additional columns for sentiment polarity and subjectivity.
    """
    processed_dataframes = {}

    for key, df in dataframes.items():
        # Ensure the column exists
        if text_column not in df.columns:
            raise ValueError(f"Column '{text_column}' not found in DataFrame '{key}'.")
        
        # Copy DataFrame to avoid modifying the original
        df_copy = df.copy()

        # Apply TextBlob sentiment analysis
        df_copy['polarity'] = df_copy[text_column].apply(lambda x: TextBlob(x).sentiment.polarity)
        df_copy['subjectivity'] = df_copy[text_column].apply(lambda x: TextBlob(x).sentiment.subjectivity)

        # Store the processed DataFrame
        processed_dataframes[key] = df_copy
    
    return processed_dataframes


In [45]:
# Run sentiment analysis on the 'cleaned_post' column
sentiment_results = sentiment_analysis(processed_dfs, 'cleaned_post')

# Access results
for sheet_name, df in sentiment_results.items():
    print(f"Sheet: {sheet_name}")
    print(df[['cleaned_post', 'polarity', 'subjectivity']].head())

Sheet: sentiments\absa_sentiments
                                        cleaned_post  polarity  subjectivity
0                                         absa never      0.00         0.000
1  want loan go write energy commission cert pay ...      0.00         0.000
2  jobcentergh offer fast efficient hiring soluti...      0.30         0.750
3  amb price dropped swine buy keep coming great ...      0.80         0.750
4  imitating access bank method great least give ...      0.25         0.575
Sheet: sentiments\access_bank_sentiments
                                        cleaned_post  polarity  subjectivity
0  people send give money made sent money account...  0.000000      0.000000
1                bought 21gh airtime hasnt reflected  0.000000      0.000000
2  reporting fraudster deceiving people job recru...  0.600000      0.900000
3  ive spent close 2 hour new town branch withdra...  0.018182      0.277273
4                            please work name change  0.000000      0.000000
S

In [46]:
import os

# Folder path where you want to save the files
output_folder = "File path"  # Replace with the actual path

# Ensure the folder exists, create it if it doesn't
os.makedirs(output_folder, exist_ok=True)

# Define a function to sanitize sheet names and file names
def sanitize_name(name):
    # Replace invalid characters with an underscore or remove them
    sanitized_name = re.sub(r'[\\/:*?"<>|]', '_', name)
    return sanitized_name

# Loop through each DataFrame in the vader_results dictionary
for sheet_name, df in sentiment_results.items():
    # Sanitize the sheet name to remove invalid characters
    sanitized_sheet_name = sanitize_name(sheet_name)
    
    # Construct the file path for each DataFrame (using sanitized sheet name)
    output_file_path = os.path.join(output_folder, f"{sanitized_sheet_name}_sentiment_results.xlsx")

    # Save the DataFrame to Excel with the sanitized sheet name
    df.to_excel(output_file_path, index=False, sheet_name=sanitized_sheet_name)

    print(f"Results for {sheet_name} have been saved to {output_file_path}.")

Results for sentiments\absa_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_absa_sentiments_sentiment_results.xlsx.
Results for sentiments\access_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_access_bank_sentiments_sentiment_results.xlsx.
Results for sentiments\adb_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_adb_sentiments_sentiment_results.xlsx.
Results for sentiments\calbank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_calbank_sentiments_sentiment_results.xlsx.
Results for sentiments\cbg_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_cbg_sentiments_sentimen

C:\Users\User\.conda\envs\creating_an_environment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Results for sentiments\gcb_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_gcb_sentiments_sentiment_results.xlsx.
Results for sentiments\gtbank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_gtbank_sentiments_sentiment_results.xlsx.
Results for sentiments\stanbic_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_stanbic_bank_sentiments_sentiment_results.xlsx.
Results for sentiments\standard_chartered_bank_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentiments\sentiments_standard_chartered_bank_sentiments_sentiment_results.xlsx.
Results for sentiments\uba_sentiments have been saved to C:/Users/User/Desktop/stanbic/competitor analysis/sentiments/text blob analysed sentime